<a href="https://colab.research.google.com/github/HuinaGabriela/Image-Based-Product-Recommender/blob/main/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#   Este arquivo: Filtra o DataFrame df para 4 categorias escolhidas
#   Seleciona 200 imagens por categoria
#   Copia as imagens para data/filtrado_200/<categoria>/
#   cria o drive e salva o arquivo CSV

In [ ]:
# Bibliotecas
from google.colab import drive
import os
import shutil
import pandas as pd
from tqdm.notebook import tqdm
